In [44]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [45]:
#  0. 라이브러리 임포트 및 API 키 입력
from openai import OpenAI
import os

#  사용자에게 키 입력받기
api_key = input(" OpenAI Key를 입력하세요: ").strip()

#  클라이언트 초기화
client = OpenAI(api_key=api_key)


In [46]:
# 1. train.jsonl 파일 업로드
train_file = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune"
)
print("OK! Train File ID:", train_file.id)


OK! Train File ID: file-ByLZDjnv8L4qgxQBczLFBy


In [47]:
# 2. test.jsonl 파일 업로드 (선택)
val_file = client.files.create(
    file=open("test.jsonl", "rb"),
    purpose="fine-tune"
)
print("OK! Validation File ID:", val_file.id)


OK! Validation File ID: file-LKjYC5kJroNi9fHPBdkiE5


In [48]:
# 3. 파인튜닝 시작
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=val_file.id,
    model="gpt-3.5-turbo",
    suffix="family-diary"
)

print("OK! 파인튜닝 Job ID:", fine_tune_job.id)


OK! 파인튜닝 Job ID: ftjob-CLdPaXVbAwMPO8zbGRzffl2r


In [49]:
# 4. 파인튜닝 상태 확인
job_id = fine_tune_job.id
status = client.fine_tuning.jobs.retrieve(job_id)
print("OK! 현재 상태:", status.status)


OK! 현재 상태: validating_files


In [50]:
# 가장 최근 파인튜닝 job 정보 확인
jobs = client.fine_tuning.jobs.list(limit=10)

for job in jobs.data:
    print("🔹 모델 이름:", job.fine_tuned_model)


🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: None
🔹 모델 이름: ft:gpt-3.5-turbo-0125:personal:family-diary:Bedwhlqk
🔹 모델 이름: ft:gpt-3.5-turbo-0125:personal:family-diary:BedsaIAa


In [51]:
# 5. 파인튜닝 완료 후, 모델로 테스트
completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:family-diary:Bedwhlqk",  # 실제 완성된 모델명으로...
    messages=[
        {"role":"user","content":"엄마: 애들은 다들 일어났니?\n엄마: 엄마는 병원 왔어.\n엄마: 태건이는 어디야? \n태건: 학교 가는중 \n엄마: 아빠는 왜 답없대?\n태건: 아빠가 학교에 태워다 준대요.\n엄마: 웬일 태워다준대?ㅎ\n채원: 오홍\n엄마: 채원인 출근준비중?\n채원: 넹~\n채원: 서울대 연구실 가용\n엄마: 오늘도 가는겨? 혼자 가는거야?\n채원: 동기들이랑!\n엄마: ㅇㅋ\n엄마: 바람이 너무 분다. 다들 조심해~\n채원: 안날라 가게 조심할게요~"}
    ]
)

print("OK! 요약 결과:", completion.choices[0].message.content)


OK! 요약 결과: 오늘은 엄마가 병원을 다녀왔고, 태건은 학교에 가는 길이었으며 채원은 서울대 연구실을 방문했다. 가족들끼리 위치와 일정을 공유하며 따뜻한 소통을 나눈 하루였다.
